# Tarea PySpark

### Objetivo:
 Analizar la eficiencia de los jugadores en términos generales y por posición, así como determinar la contribución al equipo por jugador tomando en cuenta los datos obtenidos

Usarás la base de datos del archivo 'fusbol.csv' para obtener tus datos. Checa la estructura del archivo para ver si es necesario limpiar la informacion, ver su estructura y así sea más fácil completar la tarea. Besos.

### Ejercicio 1:
Carga la base de datos en un DataFrame de Pyspark (con 2 nucleos). Valida los rangos de los valores donde sea aplicable, así como su corrección (en caso de ser necesaria). 
Después, utilizando las variables más relevantes como 'Ast/90', 'PassCmp%', etc., concluye qué ligas tienen los mejores jugadores por posición. Es decir, si los mejores jugadores defensas son de la liga francesa, inglesa, etc., por ejemplo.
Como cada persona tiene una definición de "mejor", utiliza las siguientes metricas por posición:
- Delanteros: npG+A/90 y npxG+xA/90
- Medios: KeyPass/90 y PassCmp%
- Defensas: PressSucc% y Interceptions/90

Con los resultados obtenidos, grafica por posición para que tu conclusión tenga un respaldo visual también.

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, when, avg, percent_rank
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#Esto me lo dijo ChatGPT
spark = SparkSession.builder.appName('tarea')\
    .config('spark.master', 'local[2]')\
    .config('spark.executor.memory', '1g')\
    .config("spark.sql.shuffle.partitions", 10)\
    .config('spark.driver.memory','1g')\
    .getOrCreate()

In [ ]:
df = spark.read.csv('fusbol.csv', header=True, inferSchema=True)

In [ ]:
df= df.dropna()

In [ ]:
df = df.withColumn('PassCmp%', when(col('PassCmp%') > 100, 100).otherwise(col('PassCmp%')))
df = df.withColumn('PassCmp%', when(col('PassCmp%') < 0, 0).otherwise(col('PassCmp%')))
df = df.withColumn('PressSucc%', when(col('PressSucc%') > 100, 100).otherwise(col('PressSucc%')))
df = df.withColumn('PressSucc%', when(col('PressSucc%') < 0, 0).otherwise(col('PressSucc%')))

### Delanteros

In [ ]:
delanteros=df.filter(col('pos').like("%FW%"))
del_met=delanteros.groupBy('Comp').agg(avg('npG+A/90').alias('prom_npG+A/90'), avg('npxG+xA/90').alias('prom_npxG+xA/90'))
del_met.show()

In [ ]:
# Extract data from DataFrame
competitions = del_met.select('Comp').rdd.flatMap(lambda x: x).collect()
prom_npG_A_90 = del_met.select('prom_npG+A/90').rdd.flatMap(lambda x: x).collect()
prom_npxG_xA_90 = del_met.select('prom_npxG+xA/90').rdd.flatMap(lambda x: x).collect()

# Plot data
plt.figure(figsize=(10, 6))
bar_width = 0.35
index = range(len(competitions))

plt.bar(index, prom_npG_A_90, bar_width, label='prom_npG+A/90')
plt.bar([i + bar_width for i in index], prom_npxG_xA_90, bar_width, label='prom_npxG+xA/90')

# Add labels and title
plt.xlabel('Liga')
plt.ylabel('Resultado')
plt.title('Métrica por cada liga')
plt.xticks([i + bar_width / 2 for i in index], competitions, rotation=45)
plt.legend()

plt.tight_layout()
plt.show()

La mejor en este caso es la it Serie A porque le va mejor en ambas variables en delanteros

### Medios

In [ ]:
medios=df.filter(col('pos').like("%MF%"))
med_met=medios.groupBy('Comp').agg(avg('KeyPass/90').alias('avg_KeyPass/90'), avg('PassCmp%').alias('avg_PassCmp%'))
med_met.show()

In [ ]:
# Extract data from DataFrame
competitions_med = med_met.select('Comp').rdd.flatMap(lambda x: x).collect()
avg_key_pass_90 = med_met.select('avg_KeyPass/90').rdd.flatMap(lambda x: x).collect()
avg_pass_cmp = med_met.select('avg_PassCmp%').rdd.flatMap(lambda x: x).collect()

# Plot data
plt.figure(figsize=(10, 6))
bar_width = 0.35
index = range(len(competitions_med))

plt.bar(index, avg_key_pass_90, bar_width, label='Average Key Passes/90')
plt.bar([i + bar_width for i in index], avg_pass_cmp, bar_width, label='Average Pass Completion %')

# Add labels and title
plt.xlabel('Competition')
plt.ylabel('Average Metric')
plt.title('Average Metrics by Competition (Midfielders)')
plt.xticks([i + bar_width / 2 for i in index], competitions_med, rotation=45)
plt.legend()

plt.tight_layout()
plt.show()

Como no se ve muy bien, usaremos otra gráfica

In [ ]:
# Extract data from DataFrame
avg_key_pass_90 = med_met.select('avg_KeyPass/90').rdd.flatMap(lambda x: x).collect()
avg_pass_cmp_percent = med_met.select('avg_PassCmp%').rdd.flatMap(lambda x: x).collect()
competitions = med_met.select('Comp').rdd.flatMap(lambda x: x).collect()

# Plot data
plt.figure(figsize=(10, 6))

plt.scatter(avg_key_pass_90, avg_pass_cmp_percent, label='Competitions')

# Add labels and title
plt.xlabel('Average KeyPass/90')
plt.ylabel('Average PassCmp%')
plt.title('Average Metrics for Midfielders by Competition')

# Add competition labels for each point
for i, txt in enumerate(competitions):
    plt.annotate(txt, (avg_key_pass_90[i], avg_pass_cmp_percent[i]), xytext=(5, -5), textcoords='offset points')

plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

Va a depender de qué nos interesa más peroestá entre la serie A y la Premier League en medios

In [ ]:
### Defensas

In [ ]:
defensas=df.filter(col('pos').like("%DF%"))
def_met=defensas.groupBy('Comp').agg(avg('PressSucc%').alias('avg_PressSucc%'), avg('Interceptions/90').alias('avg_Interceptions/90'))
def_met.show()

In [ ]:
# Extract data from DataFrame
competitions = def_met.select('Comp').rdd.flatMap(lambda x: x).collect()
avg_press_succ = def_met.select('avg_PressSucc%').rdd.flatMap(lambda x: x).collect()
avg_interceptions_90 = def_met.select('avg_Interceptions/90').rdd.flatMap(lambda x: x).collect()

# Plot data
plt.figure(figsize=(10, 6))
bar_width = 0.35
index = range(len(competitions))

plt.bar(index, avg_press_succ, bar_width, label='avg_PressSucc%')
plt.bar([i + bar_width for i in index], avg_interceptions_90, bar_width, label='avg_Interceptions/90')

# Add labels and title
plt.xlabel('Competition')
plt.ylabel('Average Metric')
plt.title('Average Metrics by Competition for Defenders')
plt.xticks([i + bar_width / 2 for i in index], competitions, rotation=45)
plt.legend()

In [ ]:

# Extract data from DataFrame
avg_press_succ = def_met.select('avg_PressSucc%').rdd.flatMap(lambda x: x).collect()
avg_interceptions_90 = def_met.select('avg_Interceptions/90').rdd.flatMap(lambda x: x).collect()
competitions = def_met.select('Comp').rdd.flatMap(lambda x: x).collect()

# Plot data
plt.figure(figsize=(10, 6))

plt.scatter(avg_press_succ, avg_interceptions_90, label='Competitions')

# Add labels and title
plt.xlabel('Average PressSucc%')
plt.ylabel('Average Interceptions/90')
plt.title('Average Metrics for Defenders by Competition')

# Add competition labels for each point
for i, txt in enumerate(competitions):
    plt.annotate(txt, (avg_press_succ[i], avg_interceptions_90[i]), xytext=(5, -5), textcoords='offset points')

plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


Aquí vemos que está entre la Bundesliga y Ligue 1 en defensas

### Ejercicio 2:

Ahora hagamos algo un poco más interesante. Escoge algún jugador de todos los disponibles y toma 5 metricas, las que quieras. Debes concluir en qué percentil se encuentra el jugador en esas métricas que escogiste. Obviamente, vas a comparar sus valores con todos los demás con los que comparte posición y liga, para no tener un sesgo y que la información no pierda robustez. Por último, genera un DataFrame de Pyspark con todos los datos solicitados. Muestra el DataFrame y conviertelo a otro de tipo pandas. Muestra los dos.

In [ ]:
jugador = df.filter(df['Player'] == "Robert Lewandowski")
jugador.show()

In [ ]:
solo_delanteros_bundesliga=df.filter((df['Comp']== 'de Bundesliga') & (col('pos').like("%FW%")))
columns_windows = [
    ('npG/90', Window.orderBy(col('npG/90').desc())),
    ('npG+A/90', Window.orderBy(col('npG+A/90').desc())),
    ('Interceptions/90', Window.orderBy(col('Interceptions/90').desc())),
    ('KeyPass/90', Window.orderBy(col('KeyPass/90').desc())),
    ('SuccDrib/90', Window.orderBy(col('SuccDrib/90').desc()))
]

# La neta no se de futbol entonces elegí las estadísticas que me dio ChatGPT
for column, window in columns_windows:
    percentile_column = f'percentile_{column}'
    solo_delanteros_bundesliga = solo_delanteros_bundesliga.withColumn(percentile_column, percent_rank().over(window))
jugador= solo_delanteros_bundesliga.filter(solo_delanteros_bundesliga['Player'] == 'Robert Lewandowski')
metricas=jugador.select('Player','percentile_npG/90', 'percentile_npG+A/90', 'percentile_Interceptions/90', 'percentile_KeyPass/90', 'percentile_SuccDrib/90')
metricas.show()